In [1]:
import sys
print(sys.version)
print(sys.executable)

3.6.7 |Anaconda, Inc.| (default, Oct 28 2018, 19:44:12) [MSC v.1915 64 bit (AMD64)]
C:\ProgramData\Anaconda3\python.exe


In [7]:
"""
本程序是多孪生体协同运行机制下的两种方法的批处理测试程序，
方法1：single metric dynamic preference method based on time window.
方法2：multi-metrics dynamic fusion method based on time window
本程序是两种方法的批处理测试程序输入的是各单孪生体的所有预测结果数据，

----------------------------------------------------------------------------------
This program is a batch test program for two methods under the multi-twin co-operation mechanism.
Method 1: single metric dynamic preference method based on time window.
Method 2: multi-metrics dynamic fusion method based on time window
This program is a batch test program for both methods. The inputs are all the prediction results data of each single twin.
"""

##导入所有必要的库，以及一些自制的函数  
#Import all necessary libraries, as well as some homebrew functions
 
%reload_ext autoreload

%autoreload 2

import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import dataloader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from pytorch_preprocessing import WindPowerTimeSeriesDataset, ComposeTransform, ToTensor

In [8]:
# 读取各单孪生体测试集预测数据
#Read the predicted data of test set for each single-digital twin 

# LSTMCNN=pd.read_csv('./保存的模型和数据/LSTMCNN_15年_预测数据.csv',low_memory=False)
# GRUCNN=pd.read_csv('./保存的模型和数据/GRUCNN_15年_预测数据.csv',low_memory=False)
# LSTM = pd.read_csv('./保存的模型和数据/LSTM_15年_预测数据.csv',low_memory=False)
# GRU = pd.read_csv('./保存的模型和数据/GRU_15年_预测数据.csv',low_memory=False)




# 因数据所有权的要求，只上传了100个点的数据作为示例
# Due to data ownership requirements, only 100 points of data were uploaded as examples
LSTMCNN=pd.read_csv('./保存的模型和数据/LSTMCNN_15年_预测数据_100.csv',low_memory=False)
GRUCNN=pd.read_csv('./保存的模型和数据/GRUCNN_15年_预测数据_100.csv',low_memory=False)
LSTM = pd.read_csv('./保存的模型和数据/LSTM_15年_预测数据_100.csv',low_memory=False)
GRU = pd.read_csv('./保存的模型和数据/GRU_15年_预测数据_100.csv',low_memory=False)

# 为不同单孪生体编号
# Numbering the different single-digital twin 

LSTMCNN.Code = "1"
GRUCNN.Code = "2"
LSTM.Code = "3"
GRU.Code = "4"

# 为不同单孪生体赋予字符串名称
# Assigning Names to Different single-digital twins

LSTMCNN.name = "LSTMCNN"
GRUCNN.name = "GRUCNN"
LSTM.name = "LSTM"
GRU.name = "GRU"





In [33]:
errors( scaler, GRU.iloc[:,1],GRU.iloc[:,2],prt = False, inv = False)


(2.1290536598556,
 8.587067786329948,
 2.9303699060579276,
 2879718.5138440323,
 0.7958656728896353,
 0.7652596655756,
 8.001445430573064)

In [14]:
# 获取归一化范围 Getting the normalized range

scaler_filename = r'./scaler.save'
scaler = joblib.load(scaler_filename)
 

# 基于时间窗口的单指标动态优选方法
#  single metric dynamic preference method based on time window.
# -----------------------------------------------------------------------------------------------

In [15]:


def errors( scaler, result ,label,prt = False, inv = False):
    ''' 通过预测值与真值，计算预测值的各项评价指标  返回了：mae,mse,rmse,mape,R2_score,mean,var
        Calculate each evaluation metric of the predicted value by comparing the 
        predicted value to the true value Returned: mae,mse,rmse,mape,R2_score,mean,var
    '''
    from sklearn.metrics import mean_squared_error #MSE
    from sklearn.metrics import mean_absolute_error #MAE
    from sklearn.metrics import r2_score#R 2
    from sklearn.metrics import mean_absolute_percentage_error #MAPE

    if inv:
        result = inverse_transform_col(scaler,result,8)
        label = inverse_transform_col(scaler,label,8)
    
    R2_score=r2_score(label,result) 
    mape = mean_absolute_percentage_error(label,result)
    mae = mean_absolute_error(label,result)
    mse = mean_squared_error(label,result)
    rmse = np.sqrt(mse)
    
    mean = np.mean(result-label)
    var = np.var(label-result)
    if prt:
        print(f"MAE: {round(mae, 4)}")
        print(f"MSE: {round(mse, 4)}")
        print(f"RMSE: {round(rmse, 4)}")
        print(f"MAPE: {round(mape, 4)}")
        print(f"R2_score: {round(R2_score, 4)}")
        print(f"mean: {round(mean, 4)}")
        print(f"var: {round(var, 4)}")
    
    return mae,mse,rmse,mape,R2_score,mean,var

def inverse_transform_col(scaler,y,n_col):
    '''
    scaler是对包含多个feature的X拟合的,y对应其中一个feature,n_col为y在X中对应的列编号.返回y的反归一化结果
    The scaler is fitted to X containing multiple features, y corresponds to one of the features,
    and n_col is the number of the column in X that y corresponds to.
    Return the inverse normalized result of y
    '''
    y = y.copy()
    y -= scaler.min_[n_col]
    y /= scaler.scale_[n_col]
    return y

    
def error_逐点_n点(dataset,scaler,df1,ind,Bat,inv = False):
    """
    Calculate the results of one multi-twin combination under Method 1（ single metric dynamic preference method based on time window.）
    
    
    Parameters:
        dataset：由不同单孪生体组合成的多孪生体输入数据集合 
                 A collection of multi-digital-twin input data combined from different single digital twins
        scaler:  归一化范围  
                 normalized range
        ind：    计算次数
                 Number of calculations
        df1：    dataframe 用于保存中间计算结果的引用
                 dataframe Used to hold references to the results of intermediate calculations
        Bat：    窗口长度      
                 Window length 
        inv：    是否反归一化
                 Whether anti-normalization

    """
    
    
    name = ""
    for i in dataset:    
        name  += i.name+ "+"
    # 输出当前dataset 组成的多孪生体名称 Outputs the name of the current dataset composed of multi-digital twin
    print(name)
    for bat in Bat:
        #print("bat",bat)
        select_result = []   #优选出的预测结果  Preferred prediction results
        select_index = []    #优选出的预测结果所属孪生体编号 Preferred twin number to which the prediction belongs
        select_label =[]     #对应的真值    The corresponding truth value 
        #print(".............",len(dataset[0]))
        for i in range(len(dataset[0])-bat-1):
            mae =[]    
            mse =[]
            rmse =[]
            mape =[]
            r2_score =[]
            mean =[]
            var = []
            for dset in dataset:

                result_1 = dset.iloc[i:i+bat,1] 
                label = dset.iloc[i:i+bat,2] 
                #print(dset.name)
               
                    
                mae_1,mse_1,rmse_1,mape_1,r2_score_1,mean_1,var_1 = errors( scaler, result_1 ,label,prt = False, inv = False)
                mae.append(mae_1)
                mse.append(mse_1)
                rmse.append(rmse_1)
                mape.append(mape_1)
                r2_score.append(r2_score_1)
                mean.append(mean_1)
                var.append(var_1)


            if i != len(dset)-bat:
                lst = rmse
                m = min(lst)   #rmse 最小值   rmse Minimum
                select_result.append(dataset[lst.index(m)].iloc[i+bat+1,1])
                select_index.append(lst.index(m))
                select_label.append(dataset[lst.index(m)].iloc[i+bat+1,2])
        #print(select_label.__len__())
        #print(select_result.__len__())
        select_result = np.array(select_result)
        select_label = np.array(select_label)
        mae_n,mse_n,rmse_n,mape_n,R2_score_n,mean_n,var_n = errors(scaler, select_result ,select_label,prt = True, inv = False)
        df1.loc[ind] = [name,bat,mae_n,mse_n,rmse_n,R2_score_n,mean_n,var_n]
        ind = ind+1
        print("第"+str(ind)+name+str(bat))
        
#         df2 = pd.DataFrame({ 
#             'select_result': select_result,
#             'label': select_label,
#             "select_index":select_label
#         })

#         df2.to_csv('./保存的模型和数据/error_'+name+str(bat)+'.csv') 
    return select_result,select_index,df1,ind
    
def errors_all_kinds_MDT(datalist,scaler):
    """
    计算窗口为Bat、不同多孪生体在方法1下的计算结果
    Calculation window for Bat, different multi-twins under method 1    
    参数：
     Parameters: 
         datalist：不同的单孪生体数组  Different single-twin arrays
         scaler：  反归一化对象    inverse normalized object 
    """
    Bat = [10]
    import pandas as pd
    from itertools import combinations
    new_list = []
    new_list+=list(combinations(datalist, 2))
    new_list+=list(combinations(datalist, 3))
    new_list+=list(combinations(datalist, 4))
    print(new_list.__len__())
    ind = 0
    df1 = pd.DataFrame(data=None,columns=['模型','窗口长度','mae','mse',"rmse","R2","mean_n","var_n"])
    for i in new_list:
        select_result,select_index,df1,ind = error_逐点_n点(i,scaler,df1,ind,Bat,inv = False)
    save_path = r'./保存的模型和数据/单指标_15年_测试不同窗口长度Bat'+str(Bat)+'.csv'
    df1.to_csv(save_path) 

    
def errors_all_windows(datalist,scaler):
    """
    计算不同窗口多孪生体datalist在方法1下的计算结果
    Calculation window for Bat, different multi-twins under method 1    
    参数：
     Parameters: 
         datalist：不同的单孪生体数组  Different single-twin arrays
         scaler：  反归一化对象    inverse normalized object 
    """
    Bat = [2,3,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40]
    import pandas as pd
    from itertools import combinations
    new_list = []
    new_list+=list(combinations(datalist, 4))
    print(new_list.__len__())
    ind = 0
    df1 = pd.DataFrame(data=None,columns=['模型','窗口长度','mae','mse',"rmse","R2","mean_n","var_n"])
    for i in new_list:
        select_result,select_index,df1,ind = error_逐点_n点(i,scaler,df1,ind,Bat,inv = False)
    save_path = r'./保存的模型和数据/单指标_15年_测试不同窗口长度Bat'+str(Bat)+'.csv'
    df1.to_csv(save_path) 
  

In [16]:
errors_all_kinds_MDT([GRU,GRUCNN,LSTM,LSTMCNN],scaler)

11
GRU+GRUCNN+
MAE: 1.9034
MSE: 6.2383
RMSE: 2.4977
MAPE: 2409240.3719
R2_score: 0.8159
mean: 0.6847
var: 5.7695
第1GRU+GRUCNN+10
GRU+LSTM+
MAE: 1.7809
MSE: 6.6387
RMSE: 2.5766
MAPE: 358732.4838
R2_score: 0.8041
mean: 0.4525
var: 6.434
第2GRU+LSTM+10
GRU+LSTMCNN+
MAE: 1.8615
MSE: 6.8643
RMSE: 2.62
MAPE: 1201624.6147
R2_score: 0.7974
mean: 0.7422
var: 6.3134
第3GRU+LSTMCNN+10
GRUCNN+LSTM+
MAE: 1.7959
MSE: 6.1936
RMSE: 2.4887
MAPE: 358732.4704
R2_score: 0.8172
mean: 0.361
var: 6.0633
第4GRUCNN+LSTM+10
GRUCNN+LSTMCNN+
MAE: 1.7911
MSE: 5.879
RMSE: 2.4247
MAPE: 1201624.2596
R2_score: 0.8265
mean: 0.5293
var: 5.5989
第5GRUCNN+LSTMCNN+10
LSTM+LSTMCNN+
MAE: 1.953
MSE: 8.0404
RMSE: 2.8356
MAPE: 358732.49
R2_score: 0.7627
mean: 0.3083
var: 7.9453
第6LSTM+LSTMCNN+10
GRU+GRUCNN+LSTM+
MAE: 1.8287
MSE: 6.4427
RMSE: 2.5382
MAPE: 358732.4964
R2_score: 0.8098
mean: 0.4262
var: 6.2611
第7GRU+GRUCNN+LSTM+10
GRU+GRUCNN+LSTMCNN+
MAE: 1.8177
MSE: 6.1193
RMSE: 2.4737
MAPE: 1201624.2827
R2_score: 0.8194
mean: 0.5928

In [17]:
errors_all_windows([GRU,GRUCNN,LSTM,LSTMCNN],scaler)

1
GRU+GRUCNN+LSTM+LSTMCNN+
MAE: 2.1295
MSE: 9.8104
RMSE: 3.1322
MAPE: 329146.5045
R2_score: 0.7718
mean: 0.2682
var: 9.7385
第1GRU+GRUCNN+LSTM+LSTMCNN+2
MAE: 2.0858
MSE: 9.355
RMSE: 3.0586
MAPE: 332574.9844
R2_score: 0.783
mean: 0.2108
var: 9.3106
第2GRU+GRUCNN+LSTM+LSTMCNN+3
MAE: 2.0818
MSE: 9.0545
RMSE: 3.0091
MAPE: 336075.7673
R2_score: 0.7905
mean: 0.214
var: 9.0087
第3GRU+GRUCNN+LSTM+LSTMCNN+4
MAE: 2.0188
MSE: 8.6116
RMSE: 2.9346
MAPE: 343303.1383
R2_score: 0.8029
mean: 0.2669
var: 8.5404
第4GRU+GRUCNN+LSTM+LSTMCNN+6
MAE: 1.8481
MSE: 6.8169
RMSE: 2.6109
MAPE: 350848.3419
R2_score: 0.8295
mean: 0.3912
var: 6.6638
第5GRU+GRUCNN+LSTM+LSTMCNN+8
MAE: 1.8287
MSE: 6.4427
RMSE: 2.5382
MAPE: 358732.4964
R2_score: 0.8098
mean: 0.4262
var: 6.2611
第6GRU+GRUCNN+LSTM+LSTMCNN+10
MAE: 1.6729
MSE: 5.3674
RMSE: 2.3168
MAPE: 366979.1548
R2_score: 0.8372
mean: 0.2795
var: 5.2893
第7GRU+GRUCNN+LSTM+LSTMCNN+12
MAE: 1.6408
MSE: 5.2166
RMSE: 2.284
MAPE: 375614.0578
R2_score: 0.843
mean: 0.2658
var: 5.1459
第8GR

#  基于时间窗口的多指标动态融合方法    
# multi-metrics dynamic fusion method based on time window

In [18]:
from DS import BaseMassFunction
# 导入基本 DS 证据理论计算工具包  
#import the Basic DS Theory of Evidence Calculator Toolkit

from DS import BaseMassFunction

def m_rmse_mae(a):
    """
    basic probability assignment of the i-th digital twin under the RMSE or MAE metric
    If the input contains 0, set the probability of this position to 1 to avoid a division by 0 error.
    """
    if 0 in a:
        index =  a.index(0) 
        print(index)
        for i in range(len(a)):
            if i == index:
                a[i] = 1
            else: 
                a[i] = 0
        print(a)
        return a
    else: 
        a = np.array(a)       
        Y =  1/a           #intermediate variable Yi 
        s = sum(Y)
        return Y/s 
    
    
def m_r2(r2_score):
    """
    basic probability assignment of the i-th digital twin under the R2 metric
    因为时间窗口长度可能设置较小，此时R2指标可能为负数，因此将R2首先通过减去最小的负数，将R2都转换是正数
    Because the length of the time window may be set smaller, at which time the R2 indicator may be negative, 
    the R2 is first converted to a positive number by subtracting the smallest negative number
    """
    max_r2 = max(r2_score)
    min_r2 = min(r2_score)
    r2_score = np.array(r2_score) 
    if min_r2 < 0:
        r2_score = r2_score - min_r2
    sum_r2_score = sum(r2_score)
    if sum_r2_score == 0:   # If it's all zeros, that means each digital twin has the same weight.
        len_r2 = len(r2_score)
        BBA_r2 = [1/len_r2] * len_r2
    else:     
        BBA_r2 = r2_score/sum(r2_score)
    return BBA_r2


def error_DS(dataset,scaler,df1,ind,Bat,ζ,inv = False):
    """
    计算固定阈值ζ 与窗口长度下方法2的计算结果
    Calculation of Fixed Threshold ζ vs. Window Length under Method 2  
    Parameters:
        dataset：由不同单孪生体组合成的多孪生体输入数据集合 
                 A collection of multi-digital-twin input data combined from different single digital twins
        scaler:  归一化范围  
                 normalized range
        ind：    计算次数
                 Number of calculations
        df1：    dataframe 用于保存中间计算结果的引用
                 dataframe Used to hold references to the results of intermediate calculations    
        inv：    是否反归一化
                 Whether anti-normalization
        ζ：  阈值
               Threshold
        Bat：    窗口长度      
                 Window length 
    """
    name = ""
    for i in dataset:    
        name  += i.name+ "+"
    # 输出当前dataset 组成的多孪生体名称 Outputs the name of the current dataset composed of multi-digital twin
    print(name)
    for bat in Bat:
        #print("bat",bat)
        select_result = []
        select_index = []
        select_label =[]

        #print(".............",len(dataset[0]))
        for i in range(len(dataset[0])-bat-1):

            mae =[]
            mse =[]
            rmse =[]
            mape =[]
            r2_score =[]
            mean =[]
            var = []
            pre = []
            #print(i)
            for dset in dataset:
                result_1 = dset.iloc[i:i+bat,1] 
                label = dset.iloc[i:i+bat,2] 
                mae_1,mse_1,rmse_1,mape_1,r2_score_1,mean_1,var_1 = errors( scaler, result_1 ,label,prt = False, inv = False)
                mae.append(mae_1)
                mse.append(mse_1)
                rmse.append(rmse_1)
                mape.append(mape_1)
                r2_score.append(r2_score_1)
                mean.append(mean_1)
                var.append(var_1)
              
            if i != len(dset)-bat:
                lst = rmse
                m = min(lst)
                
                BBA_rmse =  m_rmse_mae(rmse)  # basic probability assignment of the i-th digital twin under the RMSE
                BBA_mae =  m_rmse_mae(mae)    # basic probability assignment of the i-th digital twin under the MAE
                BBA_r2 = m_r2(r2_score)       # basic probability assignment of the i-th digital twin under the R2


                s_BBA = {}
                a_BBA = {}
                r_BBA = {}
                for b in range(len(dataset)):
                    s_BBA[dataset[b].Code] = BBA_rmse[b]
                    a_BBA[dataset[b].Code] = BBA_mae[b]
                    r_BBA[dataset[b].Code] = BBA_r2[b]
            
                s_B = BaseMassFunction(s_BBA)
                a_B = BaseMassFunction(a_BBA)
                r_B = BaseMassFunction(r_BBA)

                ds = s_B & a_B & r_B        #  DS 证据理论计算融合结果   DS Evidence Theory to Calculate Fusion Results
                #print(ds)
                ds_name = []
                ds_v = []
                    
                for ds_set in dataset:
                    ds_name.append(ds_set.Code)
                    ds_v.append(ds[ds_set.Code])
                #print(ds_name)
                #print(ds_v)
                ds_var = np.var(ds_v)          #计算方差  calculate the variance 
                #print(ds_var)
                
                if ds_var>ζ:
                    """
                    方差大于阈值ζ将最大值置1  其余置0    
                    when the variance is greater than the threshold ζ
                    sets the probability associated with the maximum value in m(DTi) to 1
                    while setting all other probabilities to 0   
                    """   
                    for ds_set in dataset:
                        ds[ds_set.Code] = 0 
                    ds[ds_name[ds_v.index(max(ds_v))]]=1
            
                #print(ds)
                result_ds = 0
                for ds_set in dataset:
                    result_ds += ds_set.iloc[i+bat+1,1]*ds[ds_set.Code]
                    if ds[ds_set.Code] >1:
                        print("result_1",result_1)
                        print("label",label)
                        print("r2_score",r2_score)
                        print("BBA_rmse",BBA_rmse)
                        print("BBA_mae",BBA_mae)
                        print("BBA_r2",BBA_r2)
                        print("s_BBA",s_BBA)
                        print("a_BBA",a_BBA)
                        print("r_BBA",r_BBA)
                        print("s_B",s_B)
                        print("a_B",a_B)
                        print("r_B",r_B)
                        print(ds)
 
                
                select_result.append(result_ds)
                
                select_label.append(dataset[lst.index(m)].iloc[i+bat+1,2])

        select_result = np.array(select_result)
        select_label = np.array(select_label)
        mae_n,mse_n,rmse_n,mape_n,R2_score_n,mean_n,var_n = errors(scaler, select_result ,select_label,prt = True, inv = False)
        df1.loc[ind] = [name,bat,mae_n,mse_n,rmse_n,R2_score_n,mean_n,var_n]
        ind = ind+1
        print("第"+str(ind)+name+str(bat))   
#         df2 = pd.DataFrame({ 
#             'select_result': select_result,
#             'label': select_label,
#         })

        #df2.to_csv('./保存的模型和数据2/error_DS_阈值0'+name+'.csv') 
    return select_result,df1,ind,

def errors_DS_all_kinds_MDT(datalist,scaler):
    #Bat = [1,2,3,4,6,8,10,12,14,16,18,20,30,40]
    Bat = [10]
    ζ = 0.03
    import pandas as pd
    from itertools import combinations
    new_list = []
    new_list+=list(combinations(datalist, 2))
    new_list+=list(combinations(datalist, 3))
    new_list+=list(combinations(datalist, 4))
    print(new_list.__len__())
    ind = 0
    df1 = pd.DataFrame(data=None,columns=['模型','窗口长度','mae','mse',"rmse","R2","mean_n","var_n"])
    for i in new_list:
        select_result,df1,ind = error_DS(i,scaler,df1,ind,Bat,ζ,inv = False)
    save_path = r'./保存的模型和数据/多指标_15年_'+str(Bat)+str(ζ)+'.csv'
    df1.to_csv(save_path) 

    
def errors_DS_ζ_Bat(datalist,scaler):
    import pandas as pd
    from itertools import combinations
    ζ = [0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.2]
    Bat=[5,10,15,20]
    ind = 0
    df1 = pd.DataFrame(data=None,columns=['模型','ζ','mae','mse',"rmse","R2","mean_n","var_n"])
    for i in ζ:
        select_result,df1,ind = error_DS(datalist,scaler,df1,ind,inv = False,ζ=i,Bat=Bat)
    df1.to_csv('./保存的模型和数据/error_DS_不同阈值_比较_窗口长度5.csv') 
   
    
 


In [19]:
errors_DS_all_kinds_MDT([GRU,GRUCNN,LSTM,LSTMCNN],scaler)

11
GRU+GRUCNN+
MAE: 1.9197
MSE: 6.1533
RMSE: 2.4806
MAPE: 2409240.5467
R2_score: 0.8184
mean: 0.7147
var: 5.6425
第1GRU+GRUCNN+10
GRU+LSTM+
MAE: 1.7899
MSE: 6.4609
RMSE: 2.5418
MAPE: 358732.653
R2_score: 0.8093
mean: 0.4501
var: 6.2583
第2GRU+LSTM+10
GRU+LSTMCNN+
MAE: 1.8879
MSE: 7.2548
RMSE: 2.6935
MAPE: 1201624.4779
R2_score: 0.7859
mean: 0.7478
var: 6.6956
第3GRU+LSTMCNN+10
GRUCNN+LSTM+
MAE: 1.7577
MSE: 5.9636
RMSE: 2.4421
MAPE: 358732.5266
R2_score: 0.824
mean: 0.3393
var: 5.8485
第4GRUCNN+LSTM+10
GRUCNN+LSTMCNN+
MAE: 1.8272
MSE: 6.3547
RMSE: 2.5208
MAPE: 1201624.2545
R2_score: 0.8124
mean: 0.5458
var: 6.0568
第5GRUCNN+LSTMCNN+10
LSTM+LSTMCNN+
MAE: 1.916
MSE: 7.392
RMSE: 2.7188
MAPE: 358732.5273
R2_score: 0.7818
mean: 0.2729
var: 7.3175
第6LSTM+LSTMCNN+10
GRU+GRUCNN+LSTM+
MAE: 1.7982
MSE: 6.1029
RMSE: 2.4704
MAPE: 358732.6657
R2_score: 0.8199
mean: 0.4471
var: 5.903
第7GRU+GRUCNN+LSTM+10
GRU+GRUCNN+LSTMCNN+
MAE: 1.86
MSE: 6.3699
RMSE: 2.5239
MAPE: 1201624.3949
R2_score: 0.812
mean: 0.6212

In [20]:
errors_DS_ζ_Bat([GRU,GRUCNN,LSTM,LSTMCNN],scaler)

GRU+GRUCNN+LSTM+LSTMCNN+
MAE: 2.0283
MSE: 8.6371
RMSE: 2.9389
MAPE: 339651.0056
R2_score: 0.8005
mean: 0.2366
var: 8.5812
第1GRU+GRUCNN+LSTM+LSTMCNN+5
MAE: 1.8178
MSE: 6.2957
RMSE: 2.5091
MAPE: 358732.5022
R2_score: 0.8142
mean: 0.4152
var: 6.1233
第2GRU+GRUCNN+LSTM+LSTMCNN+10
MAE: 1.6818
MSE: 5.3522
RMSE: 2.3135
MAPE: 380085.6669
R2_score: 0.8362
mean: 0.3212
var: 5.249
第3GRU+GRUCNN+LSTM+LSTMCNN+15
MAE: 1.5425
MSE: 4.6969
RMSE: 2.1672
MAPE: 404141.7156
R2_score: 0.8295
mean: 0.4152
var: 4.5244
第4GRU+GRUCNN+LSTM+LSTMCNN+20
GRU+GRUCNN+LSTM+LSTMCNN+
MAE: 2.0322
MSE: 8.7338
RMSE: 2.9553
MAPE: 339651.0054
R2_score: 0.7983
mean: 0.2323
var: 8.6798
第5GRU+GRUCNN+LSTM+LSTMCNN+5
MAE: 1.8194
MSE: 6.3345
RMSE: 2.5168
MAPE: 358732.6185
R2_score: 0.813
mean: 0.4604
var: 6.1225
第6GRU+GRUCNN+LSTM+LSTMCNN+10
MAE: 1.6367
MSE: 5.1108
RMSE: 2.2607
MAPE: 380085.7119
R2_score: 0.8436
mean: 0.3733
var: 4.9714
第7GRU+GRUCNN+LSTM+LSTMCNN+15
MAE: 1.5663
MSE: 4.9153
RMSE: 2.217
MAPE: 404141.8088
R2_score: 0.8216
m

MAE: 1.6833
MSE: 5.4946
RMSE: 2.344
MAPE: 781465.8298
R2_score: 0.8319
mean: 0.4365
var: 5.3041
第63GRU+GRUCNN+LSTM+LSTMCNN+15
MAE: 1.568
MSE: 4.8005
RMSE: 2.191
MAPE: 811340.7668
R2_score: 0.8258
mean: 0.4799
var: 4.5702
第64GRU+GRUCNN+LSTM+LSTMCNN+20
GRU+GRUCNN+LSTM+LSTMCNN+
MAE: 2.0609
MSE: 8.8396
RMSE: 2.9731
MAPE: 692563.7921
R2_score: 0.7958
mean: 0.2629
var: 8.7705
第65GRU+GRUCNN+LSTM+LSTMCNN+5
MAE: 1.8379
MSE: 6.4381
RMSE: 2.5373
MAPE: 704642.0913
R2_score: 0.81
mean: 0.4822
var: 6.2055
第66GRU+GRUCNN+LSTM+LSTMCNN+10
MAE: 1.6833
MSE: 5.4946
RMSE: 2.344
MAPE: 781465.8298
R2_score: 0.8319
mean: 0.4365
var: 5.3041
第67GRU+GRUCNN+LSTM+LSTMCNN+15
MAE: 1.568
MSE: 4.8005
RMSE: 2.191
MAPE: 811340.7668
R2_score: 0.8258
mean: 0.4799
var: 4.5702
第68GRU+GRUCNN+LSTM+LSTMCNN+20
GRU+GRUCNN+LSTM+LSTMCNN+
MAE: 2.0609
MSE: 8.8396
RMSE: 2.9731
MAPE: 692563.7921
R2_score: 0.7958
mean: 0.2629
var: 8.7705
第69GRU+GRUCNN+LSTM+LSTMCNN+5
MAE: 1.8379
MSE: 6.4381
RMSE: 2.5373
MAPE: 704642.0913
R2_score: 0.81
m

In [68]:
MAE1 =[2.6466,2.659,2.6263,2.5423]
RMSE1 = [4.6799,4.5664,4.5088,4.5663]
R21=[0.8839,0.8895,0.8923,0.8895]


MAE2 = 2.4383
RMSE2 = 4.3708
R22 = 0.8989


MAE3 =2.3874
RMSE3 = 4.3298
R23=0.9008

In [69]:
avg = 0

for i in MAE1:
    avg +=((i- MAE2)/i)
    print(f"{round((i- MAE2)/i, 4)}")
    print(f"---{round((i- MAE2), 4)}")
    
print("--------------------------")
for i in MAE1:
    avg +=((i- MAE3)/i)

    print(f"{round((i- MAE3)/i, 4)}")
    print(f"---{round((i- MAE3), 4)}")
    
print("--------------------------")
print(f"{round(avg/8, 4)}")


0.0787
---0.2083
0.083
---0.2207
0.0716
---0.188
0.0409
---0.104
--------------------------
0.0979
---0.2592
0.1021
---0.2716
0.091
---0.2389
0.0609
---0.1549
--------------------------
0.0783


In [70]:
avg = 0

for i in RMSE1:
    avg +=((i- RMSE2)/i)
    print(f"{round((i- RMSE2)/i, 4)}")
    print(f"---{round((i- RMSE2), 4)}")

print("--------------------------")

for i in RMSE1:
    avg +=((i- RMSE3)/i)
    print(f"{round((i- RMSE3)/i, 4)}")
    print(f"---{round((i- RMSE3), 4)}")
print("--------------------------")
print(f"{round(avg/8, 4)}")

0.066
---0.3091
0.0428
---0.1956
0.0306
---0.138
0.0428
---0.1955
--------------------------
0.0748
---0.3501
0.0518
---0.2366
0.0397
---0.179
0.0518
---0.2365
--------------------------
0.0501


In [71]:
avg = 0

for i in R21:
    avg +=((R22- i)/i)
    print(f"{round(( R22-i)/i, 4)}")
    print(f"绝对{round(( R22-i), 4)}")
print("--------------------------")
    
for i in R21:  
    avg +=(( R23 - i)/i)
    print(f"{round(( R23-i)/i, 4)}")
    print(f"绝对{round(( R23-i), 4)}")
    
print("--------------------------")
print(f"{round(avg/8, 4)}")

0.017
绝对0.015
0.0106
绝对0.0094
0.0074
绝对0.0066
0.0106
绝对0.0094
--------------------------
0.0191
绝对0.0169
0.0127
绝对0.0113
0.0095
绝对0.0085
0.0127
绝对0.0113
--------------------------
0.0124


In [72]:
 
MAE1 =[2.810126236,2.802831825,2.757338141,2.770465046]
RMSE1 = [5.623651043,5.496845509,5.555828501,5.596502391]
R21=[0.934932223,0.937833516,0.936492224,0.935558947]

 
MAE2 = 2.654629065
RMSE2 = 5.454767854
R22 = 0.93899634


MAE3 =2.63793139
RMSE3 = 5.457610574
R23=0.938846399

In [73]:
avg = 0

for i in MAE1:
    avg +=((i- MAE2)/i)
    print(f"{round((i- MAE2)/i, 4)}")
    print(f"---{round((i- MAE2), 4)}")
    
print("--------------------------")
for i in MAE1:
    avg +=((i- MAE3)/i)

    print(f"{round((i- MAE3)/i, 4)}")
    print(f"---{round((i- MAE3), 4)}")
    
print("--------------------------")
print(f"{round(avg/8, 4)}")


0.0553
---0.1555
0.0529
---0.1482
0.0372
---0.1027
0.0418
---0.1158
--------------------------
0.0613
---0.1722
0.0588
---0.1649
0.0433
---0.1194
0.0478
---0.1325
--------------------------
0.0498


In [74]:
avg = 0

for i in RMSE1:
    avg +=((i- RMSE2)/i)
    print(f"{round((i- RMSE2)/i, 4)}")
    print(f"---{round((i- RMSE2), 4)}")

print("--------------------------")

for i in RMSE1:
    avg +=((i- RMSE3)/i)
    print(f"{round((i- RMSE3)/i, 4)}")
    print(f"---{round((i- RMSE3), 4)}")
print("--------------------------")
print(f"{round(avg/8, 4)}")

0.03
---0.1689
0.0077
---0.0421
0.0182
---0.1011
0.0253
---0.1417
--------------------------
0.0295
---0.166
0.0071
---0.0392
0.0177
---0.0982
0.0248
---0.1389
--------------------------
0.02


In [75]:
avg = 0

for i in R21:
    avg +=((R22- i)/i)
    print(f"{round(( R22-i)/i, 4)}")
    print(f"绝对{round(( R22-i), 4)}")
print("--------------------------")
    
for i in R21:  
    avg +=(( R23 - i)/i)
    print(f"{round(( R23-i)/i, 4)}")
    print(f"绝对{round(( R23-i), 4)}")
    
print("--------------------------")
print(f"{round(avg/8, 4)}")

0.0043
绝对0.0041
0.0012
绝对0.0012
0.0027
绝对0.0025
0.0037
绝对0.0034
--------------------------
0.0042
绝对0.0039
0.0011
绝对0.001
0.0025
绝对0.0024
0.0035
绝对0.0033
--------------------------
0.0029
